In [ ]:
from google.colab import drive #mounting google drive
drive.mount('/content/drive')

import gzip #importing necessary libraries
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import warnings
warnings.filterwarnings('ignore')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
reviews = [] #fetching 1st 15 lac entries from reviews file
with gzip.open('/content/drive/MyDrive/reviews.clean.json.gz') as f:
    head = [next(f) for x in range(1500000)]
    for l in head:
        reviews.append(eval(l))

In [ ]:
places = [] #fetching 1st 15 lac entries from places file

with gzip.open('/content/drive/MyDrive/places.clean.json.gz') as f:
  head = [next(f) for x in range(1500000)]
  for l in head:
    places.append(eval(l))

In [ ]:
data_reviews_df=pd.DataFrame() #creating data frame of the reviews file attributes
data_places_df=pd.DataFrame() #creating data frame of the places file attributes

data_reviews_df['user_id']=[x['gPlusUserId'] for x in reviews]
data_reviews_df['place_id']=[x['gPlusPlaceId'] for x in reviews]
data_reviews_df['ratings']=[x['rating'] for x in reviews]


data_places_df['place_name']=[x['name'] for x in places]
data_places_df['address']=[x['address'] for x in places]
data_places_df['gps']=[x['gps'] for x in places]
data_places_df['place_id']=[x['gPlusPlaceId'] for x in places]

In [ ]:
final_data=data_reviews_df.merge(data_places_df, on='place_id', how='inner') #final data is prepared by merging both data frames on place id

print(final_data.head(5))
print(final_data.shape) #dataframe has 721599 rows and 6 columns

                 user_id               place_id  ratings          place_name  \
0  100000013500285534661  102194128241608748649      5.0   Selale Restaurant   
1  100000021336848867366  101409858828175402384      5.0  Sakana no Hokushin   
2  100000021336848867366  107443380765688783479      5.0           八木山ベニーランド   
3  100050791909819923240  107443380765688783479      4.0           八木山ベニーランド   
4  101413910070163733438  107443380765688783479      4.0           八木山ベニーランド   

                                             address                      gps  
0  [Pınarkent Bld. Atatürk Cd. Pk:20145 Denizli M...       [37.8037, 29.2209]  
1  [1 Chome-4-1 Izumichuo, Izumi Ward, Sendai, Mi...  [38.323455, 140.882111]  
2  [Japan, 〒982-0837 Miyagi, Sendai, Taihaku Ward...   [38.24675, 140.848436]  
3  [Japan, 〒982-0837 Miyagi, Sendai, Taihaku Ward...   [38.24675, 140.848436]  
4  [Japan, 〒982-0837 Miyagi, Sendai, Taihaku Ward...   [38.24675, 140.848436]  
(721599, 6)


In [ ]:
print(len(final_data['place_id'].unique())) #these are the unique places in the final data
print(len(final_data['user_id'].unique()))  #these are the unique users in the final data

465757
386978


In [ ]:
#For Item-Item Collaborative Filtering
#no. of users>> no. of items


#Statistical Features of the Ratings
final_data.describe()

,ratings
count,721599.000000
mean,4.044534
std,1.195620
min,1.000000
25%,3.000000
50%,4.000000
75%,5.000000
max,5.000000


In [ ]:
#generating items

places_to_ratings_df=pd.DataFrame()
places_to_ratings_df['place_id']=final_data['place_id']
places_to_ratings_df['ratings']=final_data['ratings']

users_to_ratings_df=pd.DataFrame()
users_to_ratings_df['user_id']=final_data['user_id']
users_to_ratings_df['ratings']=final_data['ratings']

count_ratings_users=pd.DataFrame(users_to_ratings_df.groupby('user_id',as_index=False).count())
count_ratings_users=count_ratings_users.sort_values(by='ratings',ascending=False)
count_ratings_users.rename(columns = {'ratings':'Count'}, inplace = True)
count_ratings_users=count_ratings_users[count_ratings_users['Count']>=10]

df0=count_ratings_users.merge(final_data,on='user_id',how='inner')

count_ratings_places=pd.DataFrame(places_to_ratings_df.groupby('place_id',as_index=False).count())
count_ratings_places=count_ratings_places.sort_values(by='ratings',ascending=False)
count_ratings_places.rename(columns = {'ratings':'Count'}, inplace = True)
count_ratings_places=count_ratings_places[count_ratings_places['Count']>=10]
df1=count_ratings_places.merge(df0,on='place_id', how='inner') #I am creating a new dataframe by keeping only those users who have rated at least 10 places and keeping only those 
#places which have been rated at least by 10 users

In [ ]:
print(df1.head())

                place_id  Count_x                user_id  Count_y  ratings  \
0  118197036100848201903      213  102358706157964192105      577      5.0   
1  118197036100848201903      213  100811926915679245476      510      5.0   
2  118197036100848201903      213  102262473706616682909      303      4.0   
3  118197036100848201903      213  102013564280066334773      193      4.0   
4  118197036100848201903      213  100311310218794926064      118      5.0   

     place_name                                            address  \
0  Eiffel Tower  [Champ de Mars, 5 Avenue Anatole France, 75007...   
1  Eiffel Tower  [Champ de Mars, 5 Avenue Anatole France, 75007...   
2  Eiffel Tower  [Champ de Mars, 5 Avenue Anatole France, 75007...   
3  Eiffel Tower  [Champ de Mars, 5 Avenue Anatole France, 75007...   
4  Eiffel Tower  [Champ de Mars, 5 Avenue Anatole France, 75007...   

                    gps  
0  [48.85837, 2.294481]  
1  [48.85837, 2.294481]  
2  [48.85837, 2.294481]  
3  [48

In [ ]:
df2=df1.drop(['Count_x','Count_y','ratings','user_id'],axis=1) #this dataframe I am creating to call only place id,place name, address and gps while generating recommendations
df2=df2.drop_duplicates(subset='place_id')

In [ ]:
print(df1.shape)
print(len(df1['user_id'].unique())) #this is the no. of unique users in my data
print(len(df1['place_id'].unique())) #this is the no. of unique places in my data

(11453, 8)
3026
1750


In [ ]:
#creating the user-item interaction matrix
user_item_matrix= df1.pivot_table(index='place_id', columns='user_id', values='ratings') #creating pivot table or user-item matrix--place id's are indices and user id's are columns
#print(user_item_matrix)
user_item_matrix.shape

(1750, 3026)

In [ ]:
user_item_matrix.dropna(axis=1, how='all', inplace=True) #verifying if all the places and users are rated at least once
user_item_matrix.shape

(1750, 3026)

In [ ]:
#Normalizing the ratings
user_item_matrix_norm =user_item_matrix.apply(lambda x: x - np.nanmean(x), axis=1)
user_item_matrix_norm.head()

user_id,100000242929724813348,100001176398972193069,100002789186899500135,100002870366309555032,100003840837471130074,100005784601014694175,100005861524303470068,100006553807417850183,100008176682616801686,100008779363045501008,...,102382035298719827842,102382133916145148867,102382807835788358435,102383273599499254886,102383571253119651571,102384308706862414638,102384928776492726052,102384970580977127306,102384973977403112654,102386699617555915961
place_id,,,,,,,,,,,,,,,,,,,,,
100018779661168745263,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100049916095060220864,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100051052228329426062,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100065149333726746775,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100072485644272595732,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#Replacing NaN's by 0s
user_item_matrix_norm.fillna(0, inplace=True)
user_item_matrix_norm.head()

user_id,100000242929724813348,100001176398972193069,100002789186899500135,100002870366309555032,100003840837471130074,100005784601014694175,100005861524303470068,100006553807417850183,100008176682616801686,100008779363045501008,...,102382035298719827842,102382133916145148867,102382807835788358435,102383273599499254886,102383571253119651571,102384308706862414638,102384928776492726052,102384970580977127306,102384973977403112654,102386699617555915961
place_id,,,,,,,,,,,,,,,,,,,,,
100018779661168745263,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100049916095060220864,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100051052228329426062,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100065149333726746775,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100072485644272595732,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#Calculating Similarity using Pearson's Correlation coeffecient
item_similarity = user_item_matrix_norm.T.corr()
item_similarity.head()

place_id,100018779661168745263,100049916095060220864,100051052228329426062,100065149333726746775,100072485644272595732,100077874174158886684,100104776512106587351,100110393779020461937,100130637503418050743,100130838696633414443,...,118353971851642598304,118360212077266476387,118382625045713536224,118383021365274616433,118401183730584708191,118418639274455787853,118420416172234080253,118442906479232814516,118444775679256102191,118444988755653266157
place_id,,,,,,,,,,,,,,,,,,,,,
100018779661168745263,1.000000e+00,2.598600e-19,-2.087089e-19,-9.649932e-19,2.251955e-20,9.169049e-19,NaN,-3.368521e-19,-1.301504e-18,-5.649286e-19,...,NaN,NaN,NaN,1.906666e-19,-1.345423e-19,NaN,-1.975126e-19,3.962197e-19,NaN,-1.046153e-19
100049916095060220864,2.598600e-19,1.000000e+00,-8.485224e-20,-1.327871e-19,1.014603e-20,-8.361402e-20,NaN,4.859510e-20,-4.868861e-19,6.090410e-20,...,NaN,NaN,NaN,-1.157275e-01,8.022554e-20,NaN,6.480582e-21,-3.919840e-19,NaN,1.235706e-19
100051052228329426062,-2.087089e-19,-8.485224e-20,1.000000e+00,1.781526e-19,-8.876167e-20,-4.548668e-19,NaN,-1.622685e-19,6.730209e-19,3.259520e-20,...,NaN,NaN,NaN,-3.142316e-19,-3.958947e-20,NaN,-2.364935e-19,2.456898e-19,NaN,-1.737733e-19
100065149333726746775,-9.649932e-19,-1.327871e-19,1.781526e-19,1.000000e+00,-3.963718e-19,-1.939258e-18,NaN,1.534206e-19,2.674365e-18,7.208055e-19,...,NaN,NaN,NaN,-4.362555e-19,-4.427159e-19,NaN,-1.174261e-18,1.253611e-19,NaN,2.639016e-19
100072485644272595732,2.251955e-20,1.014603e-20,-8.876167e-20,-3.963718e-19,1.000000e+00,2.391163e-19,NaN,5.550016e-20,-4.259410e-19,-1.137610e-02,...,NaN,NaN,NaN,2.055806e-19,-2.974531e-03,NaN,-2.332336e-20,-5.134551e-20,NaN,-1.729054e-19


In [ ]:
def get_similar_place(place_id): #This function takes place id and return similar place and similarity score of that place with the given place
        sim_places = item_similarity.sort_values(by=place_id, ascending=False).index[1:]
        sim_score = item_similarity.sort_values(by=place_id, ascending=False).loc[:, place_id].tolist()[1:]
        return sim_places, sim_score

In [ ]:
places, score = get_similar_place('100072485644272595732')
for x,y in zip(places[:5], score[:5]):
    print("{} with similarity of {}".format(x, y))

105097977455458910656 with similarity of 0.36418235596271814
115614667292930305582 with similarity of 0.20628732920143172
115990545954997796025 with similarity of 0.1728046379728308
115088230679165187897 with similarity of 0.16280289290907793
100226077709382778907 with similarity of 0.16228922528048628


In [ ]:
# predict the rating of a given place by a given user
def predict_rating(user_id, place_id, max_neighbor=2):
    places, scores = get_similar_place(place_id)
    place_arr = np.array([x for x in places])
    sim_arr = np.array([x for x in scores])
    # select only the place that has already rated by given user
    filtering = user_item_matrix_norm[user_id].loc[place_arr] != 0
    # calculate the predicted score
    s = np.dot(sim_arr[filtering][:max_neighbor], user_item_matrix[user_id].loc[place_arr[filtering][:max_neighbor]])/ np.sum(sim_arr[filtering][:max_neighbor])
    return s

In [ ]:
predict_rating('100000242929724813348', "100072485644272595732")

3.0000000000000004

In [ ]:
def get_recommendation(user_id, n=5):
    predicted_rating = np.array([])
    for _place in user_item_matrix_norm.index:
        predicted_rating = np.append(predicted_rating, predict_rating(user_id, _place))
    # don't recommend something that user has already rated
    temp = pd.DataFrame({'predicted_rating':predicted_rating, 'place':user_item_matrix_norm.index})
    filtering = (user_item_matrix_norm[user_id] == 0.0)
    temp = temp.loc[filtering.values].sort_values(by='predicted', ascending=False)
    # recommend n places to user
    return df2.loc[df1['place_id'].isin(temp.place[:n])]

    

In [ ]:
s=get_recommendation('100000242929724813348')
s

,place_id,place_name,address,gps
2514,109780063811304933686,Shake Shack,"[691 8th Ave, New York, NY 10036]","[40.758543, -73.989251]"
5332,110436704643356979559,Yolk,"[1120 S Michigan Ave, Chicago, IL 60605]","[41.868863, -87.624566]"
5785,115368777014954173592,San Francisco Museum of Modern Art,"[151 3rd St, San Francisco, CA 94103]","[37.785718, -122.401051]"
8682,101812107313222854735,Clarke's Charcoal Broiler,"[615 W El Camino Real, Mountain View, CA 94040]","[37.384725, -122.082555]"
9301,101107381184663794564,Bill Speidel's Underground Tour,"[608 1st Ave, Seattle, WA 98104]","[47.602222, -122.333698]"
